# Generative Steps for novel PabB Inhibitors

This notebook contains the steps to use the scorer.py funciton plus chemsampler to sample and generate compounds optimised for PabB binding.

In [1]:
# import necessary libraries
import os
import pandas as pd 
import csv
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit import Chem
from ersilia import ErsiliaModel
from chemsampler import ChemSampler

# set paths for data, results, and source code
DATAPATH = "../data"
RESULTSPATH = "../results"
SOURCEPATH = "../src"

# activate the chemsampler environment (assumed to have been done already)
# (no code is included here as the exact method of activating the environment can vary)

### Initial dataset preparation

The reference_library.csv from https://github.com/ersilia-os/groverfeat/blob/main/data/reference_library.csv was downloaded and addded to the repo. This was filtered based loosely on Lipinski's rule of 5, but with caveats for lead compounds. More info can be found on the wiki page here https://en.wikipedia.org/wiki/Lipinski%27s_rule_of_five.


The aim of producing this dataset of ~200k lead-like compounds is to dock a large number of potential leads into PabB and obtain docking scores. Compounds that dock with a high affinity when compared to the known binders can be used alongside the known binders as a list of compounds to seed chemsampler. We are looking for 50-100 compounds to use as a seed. Chemsampler will then sample and generate ~1 million compounds. These can be filtered, remaining compounds re-docked and further generation can occur.

In [3]:
# Define the paths for the input and output files
smiles_csv = os.path.join(DATAPATH, 'smiles', 'reference_library.csv')
filtered_smiles = os.path.join(DATAPATH, 'smiles', 'testfiltered_std_smiles.csv')

# Read the SMILES strings from the CSV file into a DataFrame
df = pd.read_csv(smiles_csv, skiprows=1, header=None, names=['SMILES']) #can add skiprows=1 is there is a smiles header

# Create an empty list to store the selected SMILES strings
selected = []

# Loop through the SMILES strings and filter by molecular properties
for s in df['SMILES']:
    mol = Chem.MolFromSmiles(s)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hacceptors = Descriptors.NumHAcceptors(mol)
    hdonors = Descriptors.NumHDonors(mol)
    numrotatablebonds = Descriptors.NumRotatableBonds(mol)
    if 250 <= mw <= 450 and 1 <= logp <= 3 and hacceptors <= 5 and hdonors <= 4 and numrotatablebonds <= 5: 
        selected.append(s)

# Write the selected SMILES strings to a new CSV file
df_selected = pd.DataFrame({'SMILES': selected})
df_selected.to_csv(filtered_smiles, index=False)

In [14]:
len(df["SMILES"]) #how many original smiles?

1999381

In [24]:
len(selected) #how many in our list?

201515

In [45]:
id_list = (range(0, 201515))
len(id_list) # must create a list of 'IDs' for the ID column - this is required for the scorer.py

In [51]:
filtered_selected = os.path.join(DATAPATH, "smiles", "200k_for_chemsampler.csv")

dict = {'ID': id_list, 'SMILES': selected}  
df = pd.DataFrame(dict) 
df.to_csv(filtered_selected, index=False) 

#saves a .csv file with a column for ID and a column for SMILES which will be used to run the first round of docking into PabB.

### Chemsampler

From the docked 200k compounds the top 50 were chosen and seeded to chemsampler. This is to return a long (~1 million?) list of compounds that can be filtered for re-docking.

I also ran Chemsampler using the 2 known binders (abyssomicin C and chorismate), as a test run, to ensure the chemsampler-filtering pipeline works correctly.

Below is the code for the generation of a list of smiles, along with the code to run each sampler individually, and the code to run all of the samplers together. Alterations in the sampler.py file allow for specific samplers to be chosen.

In [2]:
# set the path to the input CSV file
input_csv = os.path.join(DATAPATH, "200k_reference_output", "102_mols_for_chemsampler.csv")
#input_csv = '/home/lyndenrooms/github/pabb-docking/data/200k_reference_output/102_mols_for_chemsampler.csv'

# set the column index to use (default is 0)
column_index = 2 # this can be changed to 2 if using the outputs of the docking, as it will contain affinity, ID and smiles columns

# read the SMILES strings from the CSV file
smiles_list = []
with open(input_csv, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # skip the header row
    for row in csv_reader:
        smiles_list.append(row[column_index])

# store the SMILES strings in a variable
smiles = smiles_list
smiles

['CC(=O)Oc1cccc(Cc2n[nH]c(=O)c3ccccc23)c1',
 'Nc1cc(OCc2cccc(Cl)c2)c2nn[nH]c2n1',
 'Cc1ccccc1CC(=O)c1cc(O)c(O)c([N+](=O)[O-])c1',
 'O=C(Cc1cccc(O)c1)N/N=C1\\C(=O)Nc2ccccc21',
 'N/C(=[NH+]\\O)c1ccnc(Oc2cccc3cnccc23)c1',
 'CC(=O)Nc1ccc(Oc2cc(N)cc([N+](=O)[O-])c2)cc1',
 'Cc1cccc(NS(=O)(=O)c2cccc3nonc23)c1',
 'O=C(Cc1ccccc1)c1cc(O)c(O)c([N+](=O)[O-])c1',
 'O=C1C(=O)N(Cc2c(F)cccc2F)c2ccc(F)cc21',
 'O=c1[nH]nc(-c2ccc(O)cc2O)n1-c1ccccc1F',
 'O=c1[nH][nH]cc1-c1ccnc(NCCc2cccc(O)c2)n1',
 'Cc1ccc(-n2cc(/C(N)=[NH+]/O)c3ccccc3c2=O)cc1',
 'CC1(CCCc2cc(=O)oc3[nH]c(=O)[nH]c(=O)c23)CC1',
 'CC(C)CCCc1cc(=O)oc2[nH]c(=O)[nH]c(=O)c12',
 'Cc1cccc(NS(=O)(=O)c2cccc([N+](=O)[O-])c2)n1',
 'O=C(O)c1cccc(/C=N/Oc2ccccc2C(=O)O)c1',
 'CCNc1ccccc1Cc1n[nH]c(=O)c2ccccc12',
 'O=C(O)c1cc(NCc2ccccc2O)ccc1O',
 'N/N=C\\c1cc(=O)oc2cc(OCc3ccccc3)ccc12',
 'Fc1cccc([C@H]2Cc3[nH]nc(-c4nnn[nH]4)c3C2)c1',
 'O=C(CCc1ccccc1)NS(=O)(=O)c1ccccc1',
 'CCCCCCc1cc(=O)oc2[nH]c(=O)[nH]c(=O)c12',
 'Nc1ccccc1Cc1n[nH]c(=O)c2ccccc12',
 'Nc1ccccc

This is for all samplers, and can tweak in sampler.py

In [3]:
# running all samplers now with biomodal and moler turned off

cs = ChemSampler()
resultsALL = cs.sample(smiles, num_samples=10000)
resultsALL

StonedSampler
Seed SMILES list: {'O=C(CCc1nc2ccccc2c(=O)[nH]1)NNC(=O)c1ccc(Cl)cc1', 'O=C(Cc1ccccn1)Nc1cccc(Cc2n[nH]c(=O)c3c2CCCC3)c1', 'Cc1ccccc1C(=O)NNS(=O)(=O)c1cccc([N+](=O)[O-])c1', 'O=C(CCc1ccccc1)NS(=O)(=O)c1ccccc1', 'O=C(c1ccc(F)cc1)c1cccc(/C=c2\\[nH]c(=O)/c(=C/c3[nH+]c[nH]c3C3CC3)[nH]c2=O)c1', 'Cc1ccccc1CC(=O)c1cc(O)c(O)c([N+](=O)[O-])c1', 'N/C(=[NH+]\\O)c1ccnc(Oc2cccc3cnccc23)c1', 'O=C(O)Cc1ccc(Nc2nc(-c3ccccc3)nc3c2C(=O)NCC3)cc1', 'Cn1c2ccccc2c(=NNc2ccc(S(=O)(=O)[NH+]=C(N)N)cc2)c2ccccc21', 'CC(C)(C)c1[nH]c[nH+]c1/C=c1\\[nH]c(=O)/c(=C/c2cccc(Oc3ccc(F)cc3)c2)[nH]c1=O'}


100%|██████████| 44/44 [00:00<00:00, 7427.73it/s]


260 molecules sampled. Selecting the best by similarity.


100%|██████████| 260/260 [00:00<00:00, 4490.65it/s]


Molecules remaining 260
260
StonedSampler
Seed SMILES list: {'COc1ccc(-c2cc(=O)n3nc(Cc4ccccc4)nc3[nH]2)cc1', 'Nc1cc(OCc2cccc(Cl)c2)c2nn[nH]c2n1', 'N#Cc1cccc(-c2cccc(C(=O)NNS(=O)(=O)c3ccccc3)c2)c1', 'O=C(c1cc(Cc2n[nH]c(=O)c3c2NCCC3)ccc1F)N1CCN(c2ccccc2)CC1', 'Cc1cc(=O)[nH]c(-c2c(C)nn(CCC(=O)N3CCc4ccccc4C3)c2C)n1', 'O=C(NCCc1ccc(Cl)cc1)C(=O)Nc1cc2c3c(c1)CC(=O)N3CCC2', 'O=C(CCC(=O)Nc1ccc2c(c1)C(=O)C(=O)NC2=O)Nc1ccc(F)cc1', 'Cn1c2ccccc2c(=NNc2ccc(S(=O)(=O)[NH+]=C(N)N)cc2)c2ccccc21', 'O=C1Nc2ccccc2/C1=N/NC(=O)C(O)(c1ccccc1)c1ccccc1', 'O=C(COc1ccc2ccccc2c1)NNC(=O)c1ccc([N+](=O)[O-])cc1'}


...

ChunkedEncodingError: search may be incomplete


100%|██████████| 10/10 [02:54<00:00, 17.49s/it]


905
['CC1=C(C)OC(C(=O)OCC(=O)NC(=O)NCC2=CC=CS2)=C1', 'CC1=CC(C(=O)NC[C@@H](C)NC(=O)CC2CC=CC2)=CC(C)=N1', 'CC1=CC(Br)=CC(C(=O)OCC(=O)NC(=O)NCC2=CC=CS2)=C1N', 'CC1=CC(C(=O)NCC(C)NC(=O)CC2CC=CC2)=CC(C)=N1', 'COCC1=CC=C(NC2=NC(C3=CC=CN=C3)=NC4=CC=CC=C24)C=C1', 'NC(=O)C1=CC(F)=CC=C1NCC2=C(I)C=CC=C2F', 'CCC1=CC=C(CCNC(=O)C(=O)NC2=CC=C3C(=C2)CCCN3C(C)=O)C=C1', 'COC1=CC=C(C2=NOC(CC3=CC=C(S(N)(=O)=O)C=C3)=N2)C=C1Cl', 'CC1(O)CCN(C(=O)C2=CC(CC3=NNC(=O)C4=CC=CC=C34)=CC=C2F)CC1', 'CC1=CC=C(C(=O)N2CCN(C(=O)C3CCCC3)CC2)C=C1NC(=O)C4=CC=CC=C4F', 'CC(=O)N1CCCC2=CC(NC(=O)C(=O)NCCC3=CC=C(F)C(C)=C3)=CC=C21', 'O=C(C1=CC(CC2=NNC(=O)C3=CC=CC=C23)=CC=C1F)N4CCSCC4', 'COC1=CC=C(C2=NOC(CC3=CC=C(S(N)(=O)=O)C=C3)=N2)C=C1F', 'CC1=CC(C2=NOC(CC3=CC=C(S(N)(=O)=O)C=C3)=N2)=CC=C1O', 'CC1=CC(C)=CC(C(=O)NCC(C)NC(=O)CC2CC=CC2)=C1', 'NS(=O)(=O)C1=CC=C(CN2C=C(C3=CC=C(Cl)C(Cl)=C3)N=N2)C=C1', 'NC1=CC=C(C=CC2=CC=CC(SCC3=NNC(=O)N3)=C2)C=C1', 'O=C(COC(=O)C1=CC(O)=CC=C1)NC(=O)NCC2=CC=CO2', 'CC1=CC=CC2=C1N=C(COC(=O)CNC(=O)C3=CC=C(Cl

102it [00:00, 126.31it/s]


{'StonedSampler': ['N=1C(CC2=CC=CC=C2)=NN3C=1NC(=CC3=O)C=4C=CC(=CC=4)O',
  'CC#S=1CC(=CC=1)C2=NC(NC=3C=CC(CC(O)=O)=CC=3)=C4C(NCCC4=N2)=O',
  'C1=C(N)N=C2C(N=NN2)=C1OCC3=CC=CC(=C3)N',
  'C1(CC2=CC=CC=C2)=NN3C(=N1)NC(=CC3=O)C4=CC=C(C)C4',
  'C1(=CC(=CC(N)=C1)OC=2C=CC(=CC=2)NC(=O)C)Br',
  'C1(=NNC(=O)C=2C1=CC=CC=2)C3=CC=C(C(=C3)C(=O)N4CCC(=O)CC4)F',
  'C1=C(F)C=CC(=C1)N=CCC(=O)NC2=CC=C3C(=O)NC(C(=O)C3=C2)=O',
  'C1(=CC=C(NC2=NC(=NC=3CCNC(C2=3)=O)C=4C=CC=CC=4)C=C1)C(O)=O',
  'C(NCC=1C=CN=CC=1)C2=CC=CC(S(=O)(=O)N3C4=CC=CC=C4CC3)=C2',
  'C=1C=C(CC)C=C(C=1OC=2C(=CC(=CC=2)F)F)O',
  'C=1C=CC=C2C(NN=C(C=12)CC3=C=C=C(F)C(=C3)C(=O)N4CCC(CC4)=O)=O',
  'C#CC(=O)NNC(CCC1=NC2=CC=CC=C2C(N1)=O)=O',
  'C1CCC=2C(=CC=C(C=2)OCC3=CC=C(C=C3)C)N4C1=NNC4=N',
  'C1=[NH1+1]C1(/C=C2/C(=O)N/C(C(N2)=O)=C\\C=3C=C(OC=4C=CC(F)=CC=4)C=CC=3)C(C)(C)N',
  'C=1C=CC=C(C=1CC=2C=3C(=CC=CC=3)C(=O)NN=2)NC',
  'C1=CC(=CC=C1)C(=O)NC2=CC=CC(CC3=NNC(C4)=C3CCCC4=O)=N2',
  'C(C1=C(C)N(CCC(=O)N2CC3=CC=CC=C3CC2)N=C1C)I',
  'C=1(C=O)C=CC

Individual samplers are below

In [ ]:
cs = ChemSampler(samplers_list= ["ChemblSampler"])
resultsCh = cs.sample(smiles, num_samples=1000)
resultsCh


In [ ]:
cs = ChemSampler(samplers_list= ["StonedSampler"])
resultsSt = cs.sample(smiles, num_samples=1000)
resultsSt


In [ ]:
cs = ChemSampler(samplers_list= ["MolerSampler"])
resultsMo = cs.sample(smiles, num_samples=1000)
resultsMo

In [ ]:
cs = ChemSampler(samplers_list= ["PubChemSampler"])
resultsPc = cs.sample(smiles, num_samples=1000)
resultsPc

In [ ]:
cs = ChemSampler(samplers_list= ["SmallWorldSampler"])
resultsSW = cs.sample(smiles, num_samples=1000)
resultsSW

In [ ]:
cs = ChemSampler(samplers_list= ["BimodalSampler"])
resultsBM = cs.sample(smiles, num_samples=1000)
resultsBM

In [ ]:
cs = ChemSampler(samplers_list= ["FasmifraSampler"])
resultsF = cs.sample(smiles, num_samples=1000)
resultsF

In [ ]:
cs = ChemSampler(samplers_list= ["MollibSampler"])
results_molib = cs.sample(smiles, num_samples=1000)
results_molib

### Filtering 

Here, I outline the steps required to filter the output of chemsampler and the docking runs.

Filters to be used:
- MW 250-450
- logP 1-4
- number of rotatable bonds <=5
- number of H bond acceptors <=5
- Number of H bond donors <=5

The above can be done simply in code.

Extra filters to apply, from Ersilia model hub:
- Retrosynthetic accessibility score (cutoff >0.5 - this score is either high or low, Y/N basically)
- Aqueous solubilty (cutoff here -4 to 4, although drugs may be in the -2 to 2 territory. Too soluble and it will be cleared, not soluble enough and it will accumulate)
- Passive permeability (The optimal log10 permeability coefficient (log Papp) for a drug will depend on the specific application and target of the drug. In general, a good log Papp value for a drug should be between -5 and -8 cm/s. Log Papp values in this range indicate that the drug is sufficiently lipophilic to cross cell membranes efficiently, but not so lipophilic that it is unable to dissolve in the watery environment of the body or that it is prone to accumulation in fatty tissues. However, it is important to note that the optimal log Papp value for a specific drug will depend on a variety of factors, including the drug's target tissue or organ, its desired pharmacokinetic profile, and its intended route of administration. Therefore, it is recommended to assess the log Papp of drugs experimentally in vitro and in vivo to determine the optimal range for a specific drug.)


In [4]:
results_list = list(set([x for k,v in resultsALL.items() for x in v])) 
results_list 
# this is turning the results in the resultsALL dictionary into a list

['c0cc(C(=O)NNC(=S)NC(=O)C(=O)NNC(=O)C(=O)NNC(=O)C(=O)NNC(=O)C(=O)O)cc([N+]([O-])=O)c0',
 'CC1(O)CCN(C(=O)C2=CC(CC3=NNC(=O)C4=CC=CC=C34)=CC=C2F)CC1',
 'CC1=CC=C(C(=O)N2CCN(C(=O)C3CCCC3)CC2)C=C1NC(=O)C4=CC=CC=C4F',
 'C=1C(=NCC=1NC(C=2C=CC=CC=2)=O)CC3=NNC(=O)C4=C3CCCC4',
 'O=C(C1=CC(CC2=NNC(=O)C3=CC=CC=C23)=CC=C1F)N4CCSCC4',
 'O(NNC(=S)NC(C=1C=CC(C(C)(C)C)=CC=1)=O)C2=CC=CC(=C2)[N+1](=O)[O-1]',
 'COC1=CC=C(C2=NOC(CC3=CC=C(S(N)(=O)=O)C=C3)=N2)C=C1F',
 'C1(=CC=C(C=C1)OC2=CC(N)=CC(=C2)[N+1]([O-1])=O)NC(=O)NC',
 'c0cc(C(=O)NNC(=S)NC(=O)c1ccc(C(C)(C)C)cc1)cc([N+]([O-])=O)c0',
 'N(C=1C=C2CC(=O)N3C2=C(C=1)CCC3)C(=O)C(=O)NCCC=O',
 'C1C=2C(=O)NN=C(C=2NCC1)CC3=CC=C(C(=C3)C(N4CCN(CC4)C5=CC=COC5)=O)F',
 'C1=C(C2=CC=NC=C2C=C1)OC=3C=C(C=CN=3)C(NO)=C=S',
 'C(CNC(=O)C(=O)NC=1C=C2C3=C(C=1)CCCN3C(C2)=O)C=CCBr',
 'C=1C=C(C(C)(C)C)C#SC=1C(NC(=S)NNC(C=2C=C([N+1]([O-1])=O)C=CC=2)=O)=O',
 'CCCC1=CC(=O)OC=2NC(=O)NC(C1=2)=O',
 'CCCC=CC(=O)NC1=CC=C(OCC2=CC=CC=C2)C=C1C',
 'C1=C(C=CC(CCNC(=O)C(=O)NC=2C=C3C4=C(CCCN4C

In [5]:
# Define the path for the output file
filtered_by_desc_output = os.path.join(DATAPATH, "200k_reference_output", "chemsampler_outputs", "r1_chemsampler_filters.csv")

# Create an empty list to store the filtered SMILES strings
filtered = []

# Loop through the results_list and filter by molecular properties
for s in results_list:
    mol = Chem.MolFromSmiles(s)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hacceptors = Descriptors.NumHAcceptors(mol)
    hdonors = Descriptors.NumHDonors(mol)
    numrotatablebonds = Descriptors.NumRotatableBonds(mol)
    if 250 <= mw <= 450 and 1 <= logp <= 4 and hacceptors <= 5 and hdonors <= 5 and numrotatablebonds <= 5: 
        filtered.append(s)

# Create a dictionary with the filtered SMILES strings
dict = {'smiles': filtered}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(dict)

# Write the DataFrame to a CSV file
df.to_csv(filtered_by_desc_output, index=False)

All Ersilia models can be run froma a single cell.

In [4]:
filtered_by_desc_ouptut = os.path.join(DATAPATH, "..", "..", "cstest", "filtered_by_desc_output.csv")

I will only be doing the RA_score as a filter, which is below:

In [8]:
# Set paths and filenames
data_path = os.path.join(DATAPATH, "200k_reference_output", "chemsampler_outputs") # this is only for this run, as the original DATAPATH will be sufficient when running the code later.
filtered_smiles_output = os.path.join(data_path, 'r1_chemsampler_filters.csv')
ra_filter_output = os.path.join(data_path, 'eos_ra_filters_output.csv')

# Load Ersilia models
ra_model = ErsiliaModel('retrosynthetic-accessibility')
sol_model = ErsiliaModel('eos6oli')
permeability_model = ErsiliaModel('eos2hbd')

# Serve models and run predictions
ra_model.serve()
input_smiles = pd.read_csv(filtered_smiles_output)['smiles']
ra_scores = ra_model.predict(input_smiles, output='pandas')
ra_model.close()

# Filter by RA score
ra_filtered = ra_scores[ra_scores['ra_score'] > 0.5]

ra_filtered.to_csv(ra_filter_output)


The below is how to run more than one Ersilia model:

In [6]:
# Set paths and filenames
data_path = os.path.join(DATAPATH, "200k_reference_output", "chemsampler_outputs") # this is only for this run, as the original DATAPATH will be sufficient when running the code later.
filtered_smiles_output = os.path.join(data_path, 'r1_chemsampler_filters.csv')
all_filters_output = os.path.join(data_path, 'eos_filters_output.csv')

# Load Ersilia models
ra_model = ErsiliaModel('retrosynthetic-accessibility')
sol_model = ErsiliaModel('eos6oli')
permeability_model = ErsiliaModel('eos2hbd')

# Serve models and run predictions
ra_model.serve()
input_smiles = pd.read_csv(filtered_smiles_output)['smiles']
ra_scores = ra_model.predict(input_smiles, output='pandas')
ra_model.close()

# Filter by RA score
ra_filtered = ra_scores[ra_scores['ra_score'] > 0.5]

sol_model.serve()
solubility = sol_model.predict(ra_filtered['input'].tolist(), output='pandas')
sol_model.close()

# Filter by solubility
sol_filtered = solubility[(solubility['solubility'] > -3) & (solubility['solubility'] < 3)]

permeability_model.serve()
permeability = permeability_model.predict(sol_filtered['input'].tolist(), output='pandas')
permeability_model.close()

# Filter by permeability
permeability_filtered = permeability[(permeability['Log10PermCoeff'] > -8) & (permeability['Log10PermCoeff'] < -4)]

# Drop unnecessary columns, rename input column, and save output to CSV
permeability_filtered.drop(columns=['Log10PermCoeff', 'key'], inplace=True)
permeability_filtered = permeability_filtered.rename(columns={'input': 'smiles'})
permeability_filtered.to_csv(all_filters_output, index=False)